In [1]:
import tensorflow as tf
import tensorflow_addons as tfa

import numpy as np
import pandas as pd
import random as python_random

from tensorflow.keras import Model
from tensorflow.keras.layers import Input, Dense, Dropout, Lambda, BatchNormalization, Activation
from tensorflow.keras.optimizers import RMSprop, Adam
from tensorflow.keras import backend as K

### y label 0,1

In [2]:
y = pd.read_csv("../cls_labelWithfea.csv")
print(len(y.loc[y["label"] == 0]))
print(len(y.loc[y["label"] == 1]))
y = y["label"]
#y.head()
#len(y[y==0])

32827
2418


### symptom（，322）

In [3]:
disA_fea_mat = pd.read_csv("disA_fea_mat.csv", index_col=0, header="infer")
disB_fea_mat = pd.read_csv("disB_fea_mat.csv", index_col=0, header="infer")

# disA_fea_mat = pd.read_csv("concat_mesh_label/con_feaA.csv",index_col=0,header="infer")
# disB_fea_mat = pd.read_csv("concat_mesh_label/con_feaB.csv",index_col=0,header="infer")
disA_fea_mat.head()

,0,1,2,3,4,5,6,7,8,9,...,312,313,314,315,316,317,318,319,320,321
43,1.646934,16.806405,21.451611,11.759815,115.787869,2.653693,93.246009,2261.805817,5.061076,18.577704,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
43,1.646934,16.806405,21.451611,11.759815,115.787869,2.653693,93.246009,2261.805817,5.061076,18.577704,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
43,1.646934,16.806405,21.451611,11.759815,115.787869,2.653693,93.246009,2261.805817,5.061076,18.577704,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
43,1.646934,16.806405,21.451611,11.759815,115.787869,2.653693,93.246009,2261.805817,5.061076,18.577704,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
43,1.646934,16.806405,21.451611,11.759815,115.787869,2.653693,93.246009,2261.805817,5.061076,18.577704,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### model

In [4]:
def euclidean_distance(vects):
    x, y = vects
    sum_square = K.sum(K.square(x - y), axis=1, keepdims=True)
    return K.sqrt(K.maximum(sum_square, K.epsilon()))

def eucl_dist_output_shape(shapes):
    shape1, shape2 = shapes
    return (shape1[0], 1)

def e_auc(y_true, y_pred):
    def euclidean_AUC(y_true,y_pred):
        #tf.compat.v1.disable_eager_execution()
        #with tf.compat.v1.Session() as sess:
            #fpr,tpr,threshold = roc_curve(y_true.eval(),y_pred.eval())
        max_pre = np.max(y_pred)
        N_y_pred = [(max_pre-i)/max_pre for i in y_pred]
        
        roc_auc = roc_auc_score(y_true,N_y_pred)
        return roc_auc

    return tf.py_function(euclidean_AUC, (y_true, y_pred),Tout=float)

def top_precision(y_true,y_pred,k):
    sorted_pred = np.sort(y_pred)
    k_threshold = sorted_pred[k-1] 
    
    y_top = [i <= k_threshold for i in y_pred] 
    
    y_true = np.array(y_true)
    y_top = np.array(y_top) #boolean
    y_true_top = y_true[y_top] 
    
    y_true_top = list(y_true_top)
    top_k_precision = y_true_top.count(1.)/k
    
    return top_k_precision
####################################################################################################################
def create_base_network_copy(input_shape):
    '''Base network to be shared (eq. to feature extraction).'''
    input = Input(shape=input_shape)
    
    x= Activation(activation='relu')(Dense(128)(input))
#     x= Activation(activation='relu')(Dense(64)(input))
    x = Dropout(0.1)(x)
    
    x= Activation(activation='relu')(Dense(64)(x))
    x = Dropout(0.1)(x)
    
    
    x= Activation(activation='relu')(Dense(64)(x))
    x = Dropout(0.1)(x)
    
    
    x= Activation(activation='relu')(Dense(32)(x))
    x = Dropout(0.1)(x)
    
#     x= Activation(activation='relu')(Dense(32)(x))
#     x = Dropout(0.1)(x)
    
    x= Activation(activation='relu')(Dense(32)(x))
    x = Dropout(0.1)(x)

    x= Activation(activation='sigmoid')(Dense(32)(x))
#     x = Dense(32)(x)
    
    return Model(input, x,name="base_net")

def create_base_network(input_shape):
    '''Base network to be shared (eq. to feature extraction).'''
    input = Input(shape=input_shape)
    
    x= Activation(activation='relu')(Dense(128)(input))
#     x= Activation(activation='relu')(Dense(64)(x))
    x = Dropout(0.1)(x)
    
    
    x= Activation(activation='relu')(Dense(64)(x))
    x = Dropout(0.1)(x)
    
    x= Activation(activation='relu')(Dense(32)(x))
    x = Dropout(0.1)(x)

#     x= Activation(activation='sigmoid')(Dense(32)(x))
    x= Activation(activation='sigmoid')(Dense(32)(x))
    return Model(input, x,name="base_net")

def create_euclLayer(input_shape):
    pair1 = Input(shape=(input_shape))
    pair2 = Input(shape=(input_shape))
              
    x = euclLayer()(pair1,pair2)
    x = Activation(activation='relu')(x)
#     x = Activation(activation='sigmoid')(x)
    
    return Model(inputs=[pair1,pair2],outputs=x,name="siamese")
    
"""state-of-the-art：input(64)-64-32-16-1"""
def create_MLP_copy(input_shape):
    
    input = Input(shape=input_shape)
    x= Activation(activation='relu')(Dense(64)(input))
#     x= Activation(activation='relu')(Dense(32)(input))
    x= Activation(activation='relu')(Dense(32)(x))
    x= Activation(activation='relu')(Dense(16)(x))
#     x= Activation(activation='relu')(Dense(8)(x))
    
#     x= Activation(activation='sigmoid',name="MLP_out")(Dense(1)(x))
    x= Dense(1,activation='sigmoid')(x)
    
    return Model(input, x ,name="mlp")

def create_MLP(input_shape):
    
    input = Input(shape=input_shape)
    x= Activation(activation='relu')(Dense(64)(input))
    
    x= Activation(activation='relu')(Dense(32)(x))
    
#     x= Activation(activation='relu')(Dense(32)(x))
    x= Activation(activation='relu')(Dense(16)(x))
    
    x= Activation(activation='relu')(Dense(8)(x))
    
#     x= Activation(activation='sigmoid',name="MLP_out")(Dense(1)(x))
    x= Dense(1,activation='sigmoid')(x)
    
    return Model(input, x ,name="mlp")

## test 1:only input symptom (siamese +mlp)

In [74]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics  import roc_curve,auc,roc_auc_score
from imblearn.under_sampling import RandomUnderSampler


cvacc,cvauc_mlp,cvpre,cvpre_top,cvrec,cvauc_sia=[],[],[],[],[],[]
i =1

"""trans to ndarray"""
y=np.array(y)
disA_fea_mat=np.array(disA_fea_mat)
disB_fea_mat=np.array(disB_fea_mat)

"""random_state=10,11,12 """
rus = RandomUnderSampler(sampling_strategy=1,random_state=11)
rus_dfmA,rus_y = rus.fit_resample(X=disA_fea_mat,y=y)
rus_dfmB,rus_y = rus.fit_resample(X=disB_fea_mat,y=y)
print('rus_y的长度:',len(rus_y))

seed = 201202
np.random.seed(seed) # This is necessary for starting Numpy generated random numbers in a well-defined initial state.
python_random.seed(seed)# This is necessary for starting core Python generated random numbers in a well-defined state.
tf.random.set_seed(seed)

"""10 fold"""
Kfold = StratifiedKFold(n_splits=10,shuffle=True,random_state=seed)
for train,test in Kfold.split(X=rus_dfmA,y=rus_y):
    ###############################################################################################
    """create model"""
    input_shape=(322)
    base_network = create_base_network(input_shape)
    
    input_a = Input(shape=input_shape)
    input_b = Input(shape=input_shape)
    # because we re-use the same instance `base_network`,the weights of the network will be shared across the two branches
    processed_a = base_network(input_a)
    processed_b = base_network(input_b)

    eucl_model = create_euclLayer(processed_a.shape[-1])
    eucl_out = eucl_model([processed_a,processed_b])   
    
    concat_fea = tf.keras.layers.concatenate([processed_a,processed_b],axis=-1)
    print("concat_fea的shape：",concat_fea.shape) #(None,32)
    MLP = create_MLP(concat_fea.shape[-1])
    MLP_out = MLP(concat_fea)
    
    model = Model(inputs=[input_a, input_b], outputs=[eucl_out,MLP_out])
    
    """L2"""
    for layer in model.layers:
        if hasattr(layer,'kernel_regularizer'):
            layer.kernel_regularizer = tf.keras.regularizers.l2(0.001)
        
    #####################################################################################################################
    
    print("-------------------------------------Kfold: {} iter-----------------------------------------".format(i))
    i+=1

    rms = RMSprop()
    callback = tf.keras.callbacks.EarlyStopping(monitor='loss',patience=5)
    
    my_loss = {"siamese":contrastive_loss,"mlp":tf.keras.losses.BinaryCrossentropy()}
    my_loss_weight = {"siamese":0,"mlp":1}
    my_metrics ={ "mlp":[tf.keras.metrics.BinaryAccuracy(),tf.keras.metrics.AUC()]} #tf.keras.metrics.Precision(top_k=len(rus_y)//100)
    
    model.compile(
#         loss= [tfa.losses.ContrastiveLoss(),tf.keras.losses.BinaryCrossentropy()],
#         loss_weights=[0.01,1],
#         metrics=[tf.keras.metrics.BinaryAccuracy(),tf.keras.metrics.AUC()],
        
        loss= my_loss,
        loss_weights=my_loss_weight,
        metrics=my_metrics,
        
        optimizer=rms,
    )
    
    # fit()中 shuffle=True
    model.fit([rus_dfmA[train],rus_dfmB[train]],[rus_y[train],rus_y[train]],
              batch_size=None,
              epochs=200,
              callbacks=callback,
              shuffle=True)
    
    scores = model.evaluate([rus_dfmA[test],rus_dfmB[test]],[rus_y[test],rus_y[test]],
                            verbose=1,
                            batch_size=len(rus_y[test]),
                            callbacks=callback)
    
    cvacc.append(scores[-2] * 100)
    cvauc_mlp.append(scores[-1] * 100)
    cvauc_sia.append(scores[-3] * 100)
    
    print("------ %s: %.2f%%\t ----- %s: %.2f%%" % 
           (model.metrics_names[-2],scores[-2]*100, model.metrics_names[-1],scores[-1]*100))
     
#print("ave_auc_keras: %.2f%% (+/- %.2f%%)\t ave_topk_pre: %.2f%% (+/- %.2f%%)\t ave_auc:%.2f%% (+/- %.2f%%)" % 
print("十折性能均值：-------- ave_acc: %.2f%% (+/- %.2f%%)\t ----- ave_auc_mlp:%.2f%% (+/- %.2f%%)" % 
      (np.mean(cvacc), np.std(cvacc),np.mean(cvauc_mlp), np.std(cvauc_mlp)))


rus_y的长度: 4836


'PYTHONHASHSEED' 不是内部或外部命令，也不是可运行的程序
或批处理文件。


concat_fea的shape： (None, 64)
-------------------------------------Kfold: 1 iter-----------------------------------------
Epoch 1/200
136/136 [==============================] - 0s 2ms/step - loss: 0.6848 - siamese_loss: 3.0520 - mlp_loss: 0.6848 - mlp_binary_accuracy: 0.5469 - mlp_auc_177: 0.5659
Epoch 2/200
136/136 [==============================] - 0s 2ms/step - loss: 0.6411 - siamese_loss: 3.2738 - mlp_loss: 0.6411 - mlp_binary_accuracy: 0.6266 - mlp_auc_177: 0.6707
Epoch 3/200
136/136 [==============================] - 0s 2ms/step - loss: 0.6094 - siamese_loss: 3.4145 - mlp_loss: 0.6094 - mlp_binary_accuracy: 0.6730 - mlp_auc_177: 0.7284
Epoch 4/200
136/136 [==============================] - 0s 2ms/step - loss: 0.5926 - siamese_loss: 3.7201 - mlp_loss: 0.5926 - mlp_binary_accuracy: 0.6870 - mlp_auc_177: 0.7449
Epoch 5/200
136/136 [==============================] - 0s 2ms/step - loss: 0.5777 - siamese_loss: 3.9795 - mlp_loss: 0.5777 - mlp_binary_accuracy: 0.7027 - mlp_auc_177: 0.7637

1/1 [==============================] - 0s 0s/step - loss: 0.5319 - siamese_loss: 4.4932 - mlp_loss: 0.5319 - mlp_binary_accuracy: 0.7459 - mlp_auc_178: 0.8161
------ mlp_binary_accuracy: 74.59%	 ----- mlp_auc_178: 81.61%
concat_fea的shape： (None, 64)
-------------------------------------Kfold: 3 iter-----------------------------------------
Epoch 1/200
136/136 [==============================] - 0s 2ms/step - loss: 0.6789 - siamese_loss: 3.2782 - mlp_loss: 0.6789 - mlp_binary_accuracy: 0.5636 - mlp_auc_179: 0.5906
Epoch 2/200
136/136 [==============================] - 0s 2ms/step - loss: 0.6431 - siamese_loss: 3.6611 - mlp_loss: 0.6431 - mlp_binary_accuracy: 0.6317 - mlp_auc_179: 0.6744
Epoch 3/200
136/136 [==============================] - 0s 2ms/step - loss: 0.6172 - siamese_loss: 3.8962 - mlp_loss: 0.6172 - mlp_binary_accuracy: 0.6599 - mlp_auc_179: 0.7162
Epoch 4/200
136/136 [==============================] - 0s 2ms/step - loss: 0.5996 - siamese_loss: 4.2007 - mlp_loss: 0.5996 - mlp_

Epoch 20/200
136/136 [==============================] - 0s 2ms/step - loss: 0.4970 - siamese_loss: 4.4716 - mlp_loss: 0.4970 - mlp_binary_accuracy: 0.7592 - mlp_auc_180: 0.8372
Epoch 21/200
136/136 [==============================] - 0s 2ms/step - loss: 0.4968 - siamese_loss: 4.5611 - mlp_loss: 0.4968 - mlp_binary_accuracy: 0.7562 - mlp_auc_180: 0.8380
Epoch 22/200
136/136 [==============================] - 0s 2ms/step - loss: 0.4982 - siamese_loss: 4.7387 - mlp_loss: 0.4982 - mlp_binary_accuracy: 0.7633 - mlp_auc_180: 0.8362
Epoch 23/200
136/136 [==============================] - 0s 2ms/step - loss: 0.4887 - siamese_loss: 4.8430 - mlp_loss: 0.4887 - mlp_binary_accuracy: 0.7668 - mlp_auc_180: 0.8435
Epoch 24/200
136/136 [==============================] - 0s 2ms/step - loss: 0.4876 - siamese_loss: 4.7489 - mlp_loss: 0.4876 - mlp_binary_accuracy: 0.7705 - mlp_auc_180: 0.8451
Epoch 25/200
136/136 [==============================] - 0s 2ms/step - loss: 0.4923 - siamese_loss: 4.7131 - mlp_los

136/136 [==============================] - 0s 2ms/step - loss: 0.4732 - siamese_loss: 5.1307 - mlp_loss: 0.4732 - mlp_binary_accuracy: 0.7677 - mlp_auc_181: 0.8547
Epoch 38/200
136/136 [==============================] - 0s 2ms/step - loss: 0.4684 - siamese_loss: 5.2435 - mlp_loss: 0.4684 - mlp_binary_accuracy: 0.7760 - mlp_auc_181: 0.8579
Epoch 39/200
136/136 [==============================] - 0s 2ms/step - loss: 0.4695 - siamese_loss: 5.2572 - mlp_loss: 0.4695 - mlp_binary_accuracy: 0.7769 - mlp_auc_181: 0.8565
Epoch 40/200
136/136 [==============================] - 0s 2ms/step - loss: 0.4716 - siamese_loss: 5.3998 - mlp_loss: 0.4716 - mlp_binary_accuracy: 0.7711 - mlp_auc_181: 0.8560
Epoch 41/200
136/136 [==============================] - 0s 2ms/step - loss: 0.4690 - siamese_loss: 5.4662 - mlp_loss: 0.4690 - mlp_binary_accuracy: 0.7771 - mlp_auc_181: 0.8578
Epoch 42/200
136/136 [==============================] - 0s 2ms/step - loss: 0.4659 - siamese_loss: 5.4346 - mlp_loss: 0.4659 - m

1/1 [==============================] - 0s 999us/step - loss: 0.5394 - siamese_loss: 5.8680 - mlp_loss: 0.5394 - mlp_binary_accuracy: 0.7293 - mlp_auc_182: 0.8184
------ mlp_binary_accuracy: 72.93%	 ----- mlp_auc_182: 81.84%
concat_fea的shape： (None, 64)
-------------------------------------Kfold: 7 iter-----------------------------------------
Epoch 1/200
137/137 [==============================] - 0s 2ms/step - loss: 0.6828 - siamese_loss: 3.4861 - mlp_loss: 0.6828 - mlp_binary_accuracy: 0.5608 - mlp_auc_183: 0.5906
Epoch 2/200
137/137 [==============================] - 0s 2ms/step - loss: 0.6371 - siamese_loss: 3.9304 - mlp_loss: 0.6371 - mlp_binary_accuracy: 0.6414 - mlp_auc_183: 0.6915
Epoch 3/200
137/137 [==============================] - 0s 2ms/step - loss: 0.6094 - siamese_loss: 4.0108 - mlp_loss: 0.6094 - mlp_binary_accuracy: 0.6678 - mlp_auc_183: 0.7279
Epoch 4/200
137/137 [==============================] - 0s 2ms/step - loss: 0.5881 - siamese_loss: 3.9900 - mlp_loss: 0.5881 - m

137/137 [==============================] - 0s 2ms/step - loss: 0.4597 - siamese_loss: 5.0177 - mlp_loss: 0.4597 - mlp_binary_accuracy: 0.7804 - mlp_auc_183: 0.8641
Epoch 45/200
137/137 [==============================] - 0s 2ms/step - loss: 0.4670 - siamese_loss: 4.9704 - mlp_loss: 0.4670 - mlp_binary_accuracy: 0.7783 - mlp_auc_183: 0.8599
Epoch 46/200
137/137 [==============================] - 0s 2ms/step - loss: 0.4592 - siamese_loss: 4.9206 - mlp_loss: 0.4592 - mlp_binary_accuracy: 0.7836 - mlp_auc_183: 0.8649
Epoch 47/200
137/137 [==============================] - 0s 2ms/step - loss: 0.4587 - siamese_loss: 4.8401 - mlp_loss: 0.4587 - mlp_binary_accuracy: 0.7785 - mlp_auc_183: 0.8651
Epoch 48/200
137/137 [==============================] - 0s 2ms/step - loss: 0.4584 - siamese_loss: 4.9888 - mlp_loss: 0.4584 - mlp_binary_accuracy: 0.7797 - mlp_auc_183: 0.8654
Epoch 49/200
137/137 [==============================] - 0s 2ms/step - loss: 0.4539 - siamese_loss: 4.8271 - mlp_loss: 0.4539 - m

137/137 [==============================] - 0s 2ms/step - loss: 0.5604 - siamese_loss: 4.3600 - mlp_loss: 0.5604 - mlp_binary_accuracy: 0.7112 - mlp_auc_185: 0.7803
Epoch 8/200
137/137 [==============================] - 0s 2ms/step - loss: 0.5481 - siamese_loss: 4.4269 - mlp_loss: 0.5481 - mlp_binary_accuracy: 0.7324 - mlp_auc_185: 0.7931
Epoch 9/200
137/137 [==============================] - 0s 2ms/step - loss: 0.5409 - siamese_loss: 4.5505 - mlp_loss: 0.5409 - mlp_binary_accuracy: 0.7393 - mlp_auc_185: 0.8012
Epoch 10/200
137/137 [==============================] - 0s 2ms/step - loss: 0.5360 - siamese_loss: 4.5372 - mlp_loss: 0.5360 - mlp_binary_accuracy: 0.7347 - mlp_auc_185: 0.8057
Epoch 11/200
137/137 [==============================] - 0s 2ms/step - loss: 0.5304 - siamese_loss: 4.8581 - mlp_loss: 0.5304 - mlp_binary_accuracy: 0.7457 - mlp_auc_185: 0.8118
Epoch 12/200
137/137 [==============================] - 0s 2ms/step - loss: 0.5300 - siamese_loss: 4.7537 - mlp_loss: 0.5300 - mlp

137/137 [==============================] - 0s 2ms/step - loss: 0.4760 - siamese_loss: 5.3614 - mlp_loss: 0.4760 - mlp_binary_accuracy: 0.7891 - mlp_auc_186: 0.8524
Epoch 50/200
137/137 [==============================] - 0s 2ms/step - loss: 0.4705 - siamese_loss: 5.4900 - mlp_loss: 0.4705 - mlp_binary_accuracy: 0.7916 - mlp_auc_186: 0.8536
Epoch 51/200
137/137 [==============================] - 0s 2ms/step - loss: 0.4720 - siamese_loss: 5.3226 - mlp_loss: 0.4720 - mlp_binary_accuracy: 0.7905 - mlp_auc_186: 0.8539
Epoch 52/200
137/137 [==============================] - 0s 2ms/step - loss: 0.4705 - siamese_loss: 5.5118 - mlp_loss: 0.4705 - mlp_binary_accuracy: 0.7887 - mlp_auc_186: 0.8528
Epoch 53/200
137/137 [==============================] - 0s 2ms/step - loss: 0.4673 - siamese_loss: 5.4948 - mlp_loss: 0.4673 - mlp_binary_accuracy: 0.7912 - mlp_auc_186: 0.8543
Epoch 54/200
137/137 [==============================] - 0s 2ms/step - loss: 0.4695 - siamese_loss: 5.5557 - mlp_loss: 0.4695 - m

### results

10-fold avg-------- ave_acc: 74.05% (+/- 1.81%)	 ----- ave_auc_mlp:81.17% (+/- 1.19%)

In [76]:
model.summary()

Model: "functional_333"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1042 (InputLayer)         [(None, 322)]        0                                            
__________________________________________________________________________________________________
input_1043 (InputLayer)         [(None, 322)]        0                                            
__________________________________________________________________________________________________
base_net (Functional)           (None, 32)           52736       input_1042[0][0]                 
                                                                 input_1043[0][0]                 
__________________________________________________________________________________________________
concatenate_166 (Concatenate)   (None, 64)           0           base_net[0][0]      

## test2:sym+mesh (siamese+mlp)
SDSP

### mesh（,24）

In [5]:
mesh_label_A = pd.read_csv("concat_mesh_label/mesh_label_A.csv",index_col=0)
print(mesh_label_A.shape)
mesh_label_A.head()

(35245, 24)


,0,1,10,11,12,13,14,15,16,17,...,21,22,23,3,4,5,6,7,8,9
"abnormalities, multiple",0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"abnormalities, multiple",0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"abnormalities, multiple",0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"abnormalities, multiple",0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"abnormalities, multiple",0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [6]:
mesh_label_B = pd.read_csv("concat_mesh_label/mesh_label_B.csv",index_col=0)

In [10]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics  import roc_curve,auc,roc_auc_score
from imblearn.under_sampling import RandomUnderSampler

cvacc,cvauc_mlp,cvpre,cvpre_top,cvrec,cvauc_sia=[],[],[],[],[],[]
i =1

y=np.array(y)
disA_fea_mat=np.array(disA_fea_mat)
disB_fea_mat=np.array(disB_fea_mat)

mesh_label_A = np.array(mesh_label_A)
mesh_label_B = np.array(mesh_label_B)

neg_to_pos=1 #2418/32827

rus = RandomUnderSampler(sampling_strategy=neg_to_pos,random_state=11)
rus_dfmA,rus_y = rus.fit_resample(X=disA_fea_mat,y=y)
rus_dfmB,rus_y = rus.fit_resample(X=disB_fea_mat,y=y)
print('rus_y的长度:',len(rus_y))
rus_mesh_A,rus_y3= rus.fit_resample(X=mesh_label_A,y=y)
rus_mesh_B,rus_y4 = rus.fit_resample(X=mesh_label_B,y=y)
print("rus_mesh_B的长度：",len(rus_mesh_B))


seed = 201202
#!PYTHONHASHSEED=0 # disable？？？
np.random.seed(seed) # This is necessary for starting Numpy generated random numbers in a well-defined initial state.
python_random.seed(seed)# This is necessary for starting core Python generated random numbers in a well-defined state.
tf.random.set_seed(seed)

Kfold = StratifiedKFold(n_splits=10,shuffle=True,random_state=seed)
for train,test in Kfold.split(X=rus_dfmA,y=rus_y):
    ###############################################################################################
    """create model"""
    input_shape=(322)
    base_network = create_base_network(input_shape)
    
    input_a = Input(shape=input_shape)
    input_b = Input(shape=input_shape)
    input_mesh_a = Input(shape=24)
    input_mesh_b = Input(shape=24)
    
    # because we re-use the same instance `base_network`,the weights of the network will be shared across the two branches
    processed_a = base_network(input_a)
    processed_b = base_network(input_b)
    
    eucl_model = create_euclLayer(processed_a.shape[-1])
    eucl_out = eucl_model([processed_a,processed_b]) 
    
    
    concat_fea = tf.keras.layers.concatenate([processed_a,input_mesh_a,processed_b,input_mesh_b],axis=-1)
    print("concat_fea的shape：",concat_fea.shape) #(None,32)
    MLP = create_MLP(concat_fea.shape[-1])
    MLP_out = MLP(concat_fea)
    
    model = Model(inputs=[input_a, input_b,input_mesh_a,input_mesh_b], outputs=[eucl_out,MLP_out])
    
    """L2"""
    for layer in model.layers:
        if hasattr(layer,'kernel_regularizer'):
            layer.kernel_regularizer = tf.keras.regularizers.l2(0.001)
        
    #####################################################################################################################

    print("-------------------------------------Kfold: {} iter-----------------------------------------".format(i))
    i+=1

    rms = RMSprop()
    callback = tf.keras.callbacks.EarlyStopping(monitor='loss',patience=5)
    
    my_loss = {"siamese":contrastive_loss,"mlp":tf.keras.losses.BinaryCrossentropy()}
    my_loss_weight = {"siamese":0,"mlp":1}
    my_metrics ={ "mlp":[tf.keras.metrics.BinaryAccuracy(),tf.keras.metrics.AUC()]} #tf.keras.metrics.Precision(top_k=len(rus_y)//100)
    
    model.compile(
#         loss= [tfa.losses.ContrastiveLoss(),tf.keras.losses.BinaryCrossentropy()],
#         loss_weights=[0.01,1],
#         metrics=[tf.keras.metrics.BinaryAccuracy(),tf.keras.metrics.AUC()],
        
        loss= my_loss,
        loss_weights=my_loss_weight,
        metrics=my_metrics,
        
        optimizer=rms,
    )
    
    # fit()中 shuffle=True
    model.fit([rus_dfmA[train],rus_dfmB[train],rus_mesh_A[train],rus_mesh_B[train]],
              [rus_y[train],rus_y[train]],
              
              batch_size=None,#默认32
              epochs=200,
              callbacks=callback,
              shuffle=True,
             )
    
    scores = model.evaluate([rus_dfmA[test],rus_dfmB[test],rus_mesh_A[test],rus_mesh_B[test]],
                            [rus_y[test],rus_y[test]],
                            verbose=1,
                            batch_size=32,
                            callbacks=callback)
    
    cvacc.append(scores[-2] * 100)
    cvauc_mlp.append(scores[-1] * 100)
    cvauc_sia.append(scores[-3] * 100)
    
    print("------ %s: %.2f%%\t ----- %s: %.2f%%" % 
           (model.metrics_names[-2],scores[-2]*100, model.metrics_names[-1],scores[-1]*100))
     
print("十折性能均值：-------- ave_acc: %.2f%% (+/- %.2f%%)\t ----- ave_auc_mlp:%.2f%% (+/- %.2f%%)" % 
      (np.mean(cvacc), np.std(cvacc),np.mean(cvauc_mlp), np.std(cvauc_mlp)))


rus_y的长度: 4836
rus_mesh_B的长度： 4836
concat_fea的shape： (None, 112)
-------------------------------------Kfold: 1 iter-----------------------------------------
Epoch 1/200
136/136 [==============================] - 0s 2ms/step - loss: 0.6709 - siamese_loss: 2.7976 - mlp_loss: 0.6709 - mlp_binary_accuracy: 0.5770 - mlp_auc_28: 0.6135
Epoch 2/200
136/136 [==============================] - 0s 2ms/step - loss: 0.6219 - siamese_loss: 2.7411 - mlp_loss: 0.6219 - mlp_binary_accuracy: 0.6528 - mlp_auc_28: 0.7037
Epoch 3/200
136/136 [==============================] - 0s 2ms/step - loss: 0.5949 - siamese_loss: 3.3611 - mlp_loss: 0.5949 - mlp_binary_accuracy: 0.6797 - mlp_auc_28: 0.7422
Epoch 4/200
136/136 [==============================] - 0s 2ms/step - loss: 0.5740 - siamese_loss: 3.6960 - mlp_loss: 0.5740 - mlp_binary_accuracy: 0.7089 - mlp_auc_28: 0.7661
Epoch 5/200
136/136 [==============================] - 0s 2ms/step - loss: 0.5659 - siamese_loss: 3.7381 - mlp_loss: 0.5659 - mlp_binary_accura

136/136 [==============================] - 0s 2ms/step - loss: 0.1751 - siamese_loss: 5.4983 - mlp_loss: 0.1751 - mlp_binary_accuracy: 0.9276 - mlp_auc_28: 0.9808
Epoch 93/200
136/136 [==============================] - 0s 2ms/step - loss: 0.1817 - siamese_loss: 5.5231 - mlp_loss: 0.1817 - mlp_binary_accuracy: 0.9297 - mlp_auc_28: 0.9794
Epoch 94/200
136/136 [==============================] - 0s 2ms/step - loss: 0.1820 - siamese_loss: 5.6906 - mlp_loss: 0.1820 - mlp_binary_accuracy: 0.9306 - mlp_auc_28: 0.9790
Epoch 95/200
136/136 [==============================] - 0s 2ms/step - loss: 0.1744 - siamese_loss: 5.7354 - mlp_loss: 0.1744 - mlp_binary_accuracy: 0.9338 - mlp_auc_28: 0.9808
Epoch 96/200
136/136 [==============================] - 0s 2ms/step - loss: 0.1832 - siamese_loss: 5.7325 - mlp_loss: 0.1832 - mlp_binary_accuracy: 0.9269 - mlp_auc_28: 0.9788
Epoch 97/200
136/136 [==============================] - 0s 2ms/step - loss: 0.1747 - siamese_loss: 5.7263 - mlp_loss: 0.1747 - mlp_bi

136/136 [==============================] - 0s 2ms/step - loss: 0.1988 - siamese_loss: 5.6634 - mlp_loss: 0.1988 - mlp_binary_accuracy: 0.9180 - mlp_auc_29: 0.9756
Epoch 83/200
136/136 [==============================] - 0s 2ms/step - loss: 0.2011 - siamese_loss: 5.6456 - mlp_loss: 0.2011 - mlp_binary_accuracy: 0.9191 - mlp_auc_29: 0.9753
Epoch 84/200
136/136 [==============================] - 0s 2ms/step - loss: 0.2063 - siamese_loss: 5.5725 - mlp_loss: 0.2063 - mlp_binary_accuracy: 0.9193 - mlp_auc_29: 0.9737
Epoch 85/200
136/136 [==============================] - 0s 2ms/step - loss: 0.1907 - siamese_loss: 5.5966 - mlp_loss: 0.1907 - mlp_binary_accuracy: 0.9216 - mlp_auc_29: 0.9778
Epoch 86/200
136/136 [==============================] - 0s 2ms/step - loss: 0.1972 - siamese_loss: 5.7423 - mlp_loss: 0.1972 - mlp_binary_accuracy: 0.9214 - mlp_auc_29: 0.9761
Epoch 87/200
136/136 [==============================] - 0s 2ms/step - loss: 0.1921 - siamese_loss: 5.7732 - mlp_loss: 0.1921 - mlp_bi

136/136 [==============================] - 0s 2ms/step - loss: 0.2343 - siamese_loss: 5.2532 - mlp_loss: 0.2343 - mlp_binary_accuracy: 0.9074 - mlp_auc_30: 0.9659
Epoch 55/200
136/136 [==============================] - 0s 2ms/step - loss: 0.2324 - siamese_loss: 5.1972 - mlp_loss: 0.2324 - mlp_binary_accuracy: 0.9051 - mlp_auc_30: 0.9669
Epoch 56/200
136/136 [==============================] - 0s 2ms/step - loss: 0.2354 - siamese_loss: 5.1732 - mlp_loss: 0.2354 - mlp_binary_accuracy: 0.8998 - mlp_auc_30: 0.9663
Epoch 57/200
136/136 [==============================] - 0s 2ms/step - loss: 0.2232 - siamese_loss: 5.2765 - mlp_loss: 0.2232 - mlp_binary_accuracy: 0.9104 - mlp_auc_30: 0.9694
Epoch 58/200
136/136 [==============================] - 0s 2ms/step - loss: 0.2150 - siamese_loss: 5.0724 - mlp_loss: 0.2150 - mlp_binary_accuracy: 0.9150 - mlp_auc_30: 0.9716
Epoch 59/200
136/136 [==============================] - 0s 2ms/step - loss: 0.2275 - siamese_loss: 5.1042 - mlp_loss: 0.2275 - mlp_bi

136/136 [==============================] - 0s 2ms/step - loss: 0.2511 - siamese_loss: 5.5255 - mlp_loss: 0.2511 - mlp_binary_accuracy: 0.9007 - mlp_auc_31: 0.9610
Epoch 69/200
136/136 [==============================] - 0s 2ms/step - loss: 0.2393 - siamese_loss: 5.5284 - mlp_loss: 0.2393 - mlp_binary_accuracy: 0.9053 - mlp_auc_31: 0.9642
Epoch 70/200
16/16 [==============================] - 0s 2ms/step - loss: 0.5333 - siamese_loss: 5.6822 - mlp_loss: 0.5333 - mlp_binary_accuracy: 0.7789 - mlp_auc_31: 0.8739    
------ mlp_binary_accuracy: 77.89%	 ----- mlp_auc_31: 87.39%
concat_fea的shape： (None, 112)
-------------------------------------Kfold: 5 iter-----------------------------------------
Epoch 1/200
136/136 [==============================] - 0s 2ms/step - loss: 0.6773 - siamese_loss: 3.1640 - mlp_loss: 0.6773 - mlp_binary_accuracy: 0.5742 - mlp_auc_32: 0.6146
Epoch 2/200
136/136 [==============================] - 0s 2ms/step - loss: 0.6364 - siamese_loss: 3.0908 - mlp_loss: 0.6364 -

...

137/137 [==============================] - 0s 2ms/step - loss: 0.5758 - siamese_loss: 3.3178 - mlp_loss: 0.5758 - mlp_binary_accuracy: 0.6952 - mlp_auc_34: 0.7685
Epoch 5/200
137/137 [==============================] - 0s 2ms/step - loss: 0.5602 - siamese_loss: 3.4514 - mlp_loss: 0.5602 - mlp_binary_accuracy: 0.7140 - mlp_auc_34: 0.7836
Epoch 6/200
137/137 [==============================] - 0s 2ms/step - loss: 0.5427 - siamese_loss: 3.8450 - mlp_loss: 0.5427 - mlp_binary_accuracy: 0.7264 - mlp_auc_34: 0.7998
Epoch 7/200
137/137 [==============================] - 0s 2ms/step - loss: 0.5354 - siamese_loss: 4.0985 - mlp_loss: 0.5354 - mlp_binary_accuracy: 0.7310 - mlp_auc_34: 0.8070
Epoch 8/200
137/137 [==============================] - 0s 2ms/step - loss: 0.5264 - siamese_loss: 4.3390 - mlp_loss: 0.5264 - mlp_binary_accuracy: 0.7335 - mlp_auc_34: 0.8137
Epoch 9/200
137/137 [==============================] - 0s 2ms/step - loss: 0.5107 - siamese_loss: 4.2755 - mlp_loss: 0.5107 - mlp_binary_

137/137 [==============================] - 0s 2ms/step - loss: 0.4739 - siamese_loss: 4.1351 - mlp_loss: 0.4739 - mlp_binary_accuracy: 0.7756 - mlp_auc_35: 0.8564
Epoch 16/200
137/137 [==============================] - 0s 2ms/step - loss: 0.4683 - siamese_loss: 3.8455 - mlp_loss: 0.4683 - mlp_binary_accuracy: 0.7785 - mlp_auc_35: 0.8586
Epoch 17/200
137/137 [==============================] - 0s 2ms/step - loss: 0.4575 - siamese_loss: 3.9969 - mlp_loss: 0.4575 - mlp_binary_accuracy: 0.7829 - mlp_auc_35: 0.8657
Epoch 18/200
137/137 [==============================] - 0s 2ms/step - loss: 0.4541 - siamese_loss: 4.2309 - mlp_loss: 0.4541 - mlp_binary_accuracy: 0.7859 - mlp_auc_35: 0.8681
Epoch 19/200
137/137 [==============================] - 0s 2ms/step - loss: 0.4445 - siamese_loss: 4.3536 - mlp_loss: 0.4445 - mlp_binary_accuracy: 0.7907 - mlp_auc_35: 0.8747
Epoch 20/200
137/137 [==============================] - 0s 2ms/step - loss: 0.4375 - siamese_loss: 4.4213 - mlp_loss: 0.4375 - mlp_bi

137/137 [==============================] - 0s 2ms/step - loss: 0.1532 - siamese_loss: 5.7128 - mlp_loss: 0.1532 - mlp_binary_accuracy: 0.9428 - mlp_auc_35: 0.9854
Epoch 108/200
137/137 [==============================] - 0s 2ms/step - loss: 0.1430 - siamese_loss: 5.7319 - mlp_loss: 0.1430 - mlp_binary_accuracy: 0.9479 - mlp_auc_35: 0.9867
Epoch 109/200
137/137 [==============================] - 0s 2ms/step - loss: 0.1530 - siamese_loss: 5.7386 - mlp_loss: 0.1530 - mlp_binary_accuracy: 0.9410 - mlp_auc_35: 0.9854
Epoch 110/200
137/137 [==============================] - 0s 2ms/step - loss: 0.1382 - siamese_loss: 5.7434 - mlp_loss: 0.1382 - mlp_binary_accuracy: 0.9444 - mlp_auc_35: 0.9877
Epoch 111/200
137/137 [==============================] - 0s 2ms/step - loss: 0.1409 - siamese_loss: 5.6515 - mlp_loss: 0.1409 - mlp_binary_accuracy: 0.9439 - mlp_auc_35: 0.9875
Epoch 112/200
137/137 [==============================] - 0s 2ms/step - loss: 0.1415 - siamese_loss: 5.6767 - mlp_loss: 0.1415 - m

137/137 [==============================] - 0s 2ms/step - loss: 0.2263 - siamese_loss: 5.0286 - mlp_loss: 0.2263 - mlp_binary_accuracy: 0.9088 - mlp_auc_36: 0.9681
Epoch 74/200
137/137 [==============================] - 0s 2ms/step - loss: 0.2242 - siamese_loss: 5.0602 - mlp_loss: 0.2242 - mlp_binary_accuracy: 0.9109 - mlp_auc_36: 0.9693
Epoch 75/200
137/137 [==============================] - 0s 2ms/step - loss: 0.2213 - siamese_loss: 4.9574 - mlp_loss: 0.2213 - mlp_binary_accuracy: 0.9120 - mlp_auc_36: 0.9693
Epoch 76/200
137/137 [==============================] - 0s 2ms/step - loss: 0.2102 - siamese_loss: 5.0227 - mlp_loss: 0.2102 - mlp_binary_accuracy: 0.9155 - mlp_auc_36: 0.9728
Epoch 77/200
137/137 [==============================] - 0s 2ms/step - loss: 0.2205 - siamese_loss: 4.9361 - mlp_loss: 0.2205 - mlp_binary_accuracy: 0.9129 - mlp_auc_36: 0.9695
Epoch 78/200
137/137 [==============================] - 0s 2ms/step - loss: 0.2231 - siamese_loss: 4.8678 - mlp_loss: 0.2231 - mlp_bi

137/137 [==============================] - 0s 2ms/step - loss: 0.2443 - siamese_loss: 5.2923 - mlp_loss: 0.2443 - mlp_binary_accuracy: 0.9008 - mlp_auc_37: 0.9632
Epoch 68/200
137/137 [==============================] - 0s 2ms/step - loss: 0.2440 - siamese_loss: 5.3979 - mlp_loss: 0.2440 - mlp_binary_accuracy: 0.9003 - mlp_auc_37: 0.9637
Epoch 69/200
137/137 [==============================] - 0s 2ms/step - loss: 0.2438 - siamese_loss: 5.2863 - mlp_loss: 0.2438 - mlp_binary_accuracy: 0.9001 - mlp_auc_37: 0.9636
Epoch 70/200
137/137 [==============================] - 0s 2ms/step - loss: 0.2420 - siamese_loss: 5.3397 - mlp_loss: 0.2420 - mlp_binary_accuracy: 0.9017 - mlp_auc_37: 0.9640
Epoch 71/200
137/137 [==============================] - 0s 2ms/step - loss: 0.2464 - siamese_loss: 5.4755 - mlp_loss: 0.2464 - mlp_binary_accuracy: 0.9033 - mlp_auc_37: 0.9627
Epoch 72/200
137/137 [==============================] - 0s 2ms/step - loss: 0.2332 - siamese_loss: 5.3362 - mlp_loss: 0.2332 - mlp_bi

### results

-------- ave_acc: 80.13% (+/- 1.24%)	 ----- ave_auc_mlp:87.90% (+/- 0.86%)

### Randomly divide 20% as test set

In [72]:
from sklearn.model_selection import StratifiedKFold,train_test_split
from sklearn.metrics  import roc_curve,auc,roc_auc_score
from imblearn.under_sampling import RandomUnderSampler


seed = 1
np.random.seed(seed) # This is necessary for starting Numpy generated random numbers in a well-defined initial state.
python_random.seed(seed)# This is necessary for starting core Python generated random numbers in a well-defined state.
tf.random.set_seed(seed)

neg_to_pos= 1
rus = RandomUnderSampler(sampling_strategy=neg_to_pos,random_state=seed) 
rus_dfmA,rus_y = rus.fit_resample(X=disA_fea_mat,y=y)
rus_dfmB,rus_y = rus.fit_resample(X=disB_fea_mat,y=y)

rus_mesh_A,rus_y = rus.fit_resample(X=mesh_label_A,y=y)
rus_mesh_B,rus_y = rus.fit_resample(X=mesh_label_B,y=y)

print('rus_y的长度:',len(rus_y))

"""split 20%"""
symA_train,symA_test,symB_train,symB_test,meshA_train,meshA_test,meshB_train,meshB_test,y_train,y_test=train_test_split(
                rus_dfmA,rus_dfmB,
                rus_mesh_A,rus_mesh_B,#mesh_label_A[-5515:-1],mesh_label_B[-5515:-1],
                rus_y,
                test_size=0.2, shuffle=True)


rus_y的长度: 4836


In [18]:
cvacc,cvauc_mlp,cvpre,cvpre_top,cvrec,cvauc_sia=[],[],[],[],[],[]
cvacc_test,cvauc_mlp_test,cvauc_sia_test=[],[],[]

###############################################################################################
"""Randomly divide the 20% test set"""
input_shape=(322)
base_network = create_base_network(input_shape)

input_a = Input(shape=input_shape)
input_b = Input(shape=input_shape)
input_mesh_a = Input(shape=24)
input_mesh_b = Input(shape=24)


processed_a = base_network(input_a)
processed_b = base_network(input_b)


eucl_model = create_euclLayer(processed_a.shape[-1])
eucl_out = eucl_model([processed_a,processed_b]) 


concat_fea = tf.keras.layers.concatenate([processed_a,input_mesh_a,processed_b,input_mesh_b],axis=-1)
print("concat_fea的shape：",concat_fea.shape) #(None,112)
MLP = create_MLP(concat_fea.shape[-1])
MLP_out = MLP(concat_fea)

model = Model(inputs=[input_a, input_b,input_mesh_a,input_mesh_b], outputs=[eucl_out,MLP_out])

"""L2"""
for layer in model.layers:
    if hasattr(layer,'kernel_regularizer'):
        layer.kernel_regularizer = tf.keras.regularizers.l2(0.001)
    
#####################################################################################################################

print("-------------------------------------Train start-----------------------------------------")

rms = RMSprop()
callback = tf.keras.callbacks.EarlyStopping(monitor='loss',patience=5)

my_loss = {"siamese":contrastive_loss,"mlp":tf.keras.losses.BinaryCrossentropy()}
my_loss_weight = {"siamese":0,"mlp":1}
my_metrics ={ "mlp":[tf.keras.metrics.BinaryAccuracy(),tf.keras.metrics.AUC()]} #tf.keras.metrics.Precision(top_k=len(rus_y)//100)

model.compile(
    loss= my_loss,
    loss_weights=my_loss_weight,
    metrics=my_metrics,
    
    optimizer=rms,
)

#symA_train,symA_test,symB_train,symB_test,meshA_train,meshA_test,meshB_train,meshB_test,y_train,y_test

# fit()中 shuffle=True
model.fit([symA_train,symB_train,meshA_train,meshB_train],
          [y_train,y_train],
          
          batch_size=None,#默认32
          epochs=200,  #200-->50-->100
          callbacks=callback,
          shuffle=True,
         )

test_scores = model.evaluate([symA_test,symB_test,meshA_test,meshB_test],
                        [y_test,y_test],
                        verbose=1,
                        batch_size=32)
#predictions[0]-siamese's outputs,predictions[1]-mlp's outputs
predictions = model.predict([symA_test,symB_test,meshA_test,meshB_test],
                        verbose=1)

cvacc_test.append(test_scores[-2] * 100)
cvauc_mlp_test.append(test_scores[-1] * 100)
cvauc_sia_test.append(test_scores[-3] * 100)


print("测试集上性能------ %s: %.2f%%\t ----- %s: %.2f%% \n" % 
           (model.metrics_names[-2],test_scores[-2]*100, model.metrics_names[-1],test_scores[-1]*100))


rus_y的长度: 4836


'PYTHONHASHSEED' 不是内部或外部命令，也不是可运行的程序
或批处理文件。


concat_fea的shape： (None, 112)
-------------------------------------Train start-----------------------------------------
Epoch 1/200
121/121 [==============================] - 0s 2ms/step - loss: 0.6712 - siamese_loss: 3.3297 - mlp_loss: 0.6712 - mlp_binary_accuracy: 0.5752 - mlp_auc_41: 0.6148
Epoch 2/200
121/121 [==============================] - 0s 2ms/step - loss: 0.6203 - siamese_loss: 3.7938 - mlp_loss: 0.6203 - mlp_binary_accuracy: 0.6569 - mlp_auc_41: 0.7136
Epoch 3/200
121/121 [==============================] - 0s 2ms/step - loss: 0.5791 - siamese_loss: 3.8642 - mlp_loss: 0.5791 - mlp_binary_accuracy: 0.6916 - mlp_auc_41: 0.7623
Epoch 4/200
121/121 [==============================] - 0s 2ms/step - loss: 0.5612 - siamese_loss: 3.9855 - mlp_loss: 0.5612 - mlp_binary_accuracy: 0.7143 - mlp_auc_41: 0.7816
Epoch 5/200
121/121 [==============================] - 0s 2ms/step - loss: 0.5492 - siamese_loss: 4.1666 - mlp_loss: 0.5492 - mlp_binary_accuracy: 0.7306 - mlp_auc_41: 0.7936
Epoch

121/121 [==============================] - 0s 2ms/step - loss: 0.1627 - siamese_loss: 5.1697 - mlp_loss: 0.1627 - mlp_binary_accuracy: 0.9400 - mlp_auc_41: 0.9833
Epoch 93/200
121/121 [==============================] - 0s 2ms/step - loss: 0.1534 - siamese_loss: 5.2542 - mlp_loss: 0.1534 - mlp_binary_accuracy: 0.9400 - mlp_auc_41: 0.9854
Epoch 94/200
121/121 [==============================] - 0s 2ms/step - loss: 0.1560 - siamese_loss: 5.1993 - mlp_loss: 0.1560 - mlp_binary_accuracy: 0.9377 - mlp_auc_41: 0.9849
Epoch 95/200
121/121 [==============================] - 0s 2ms/step - loss: 0.1582 - siamese_loss: 5.2069 - mlp_loss: 0.1582 - mlp_binary_accuracy: 0.9411 - mlp_auc_41: 0.9842
Epoch 96/200
121/121 [==============================] - 0s 2ms/step - loss: 0.1489 - siamese_loss: 5.2410 - mlp_loss: 0.1489 - mlp_binary_accuracy: 0.9423 - mlp_auc_41: 0.9862
Epoch 97/200
121/121 [==============================] - 0s 2ms/step - loss: 0.1463 - siamese_loss: 5.2543 - mlp_loss: 0.1463 - mlp_bi

测试集上性能------ mlp_binary_accuracy: 79.44%	 ----- mlp_auc_41: 85.51%

In [25]:
print(len(y_test),len(predictions),len(predictions[1]))
print(predictions[1][:10])

968 2 968
[[5.28936684e-02]
 [1.10655814e-01]
 [8.14563930e-02]
 [3.45423818e-03]
 [1.91136003e-02]
 [2.42131948e-03]
 [4.77382541e-03]
 [3.56525183e-04]
 [9.95916009e-01]
 [1.88028812e-03]]


In [65]:
AUC_data = np.array([list(y_test),[i[0] for i in predictions[1]]])
print(AUC_data.shape)
AUC_data_df=pd.DataFrame(AUC_data.T,columns=["label","predict"])
AUC_data_df.head()

(2, 968)


,label,predict
0,0.0,0.052894
1,0.0,0.110656
2,0.0,0.081456
3,1.0,0.003454
4,0.0,0.019114


In [66]:
# AUC_data_df.to_csv("auc_data_SDSP.csv")

#### 5-fold

In [73]:
cvacc,cvauc_mlp,cvpre,cvpre_top,cvrec,cvauc_sia=[],[],[],[],[],[]
seed=1
i =1

Kfold = StratifiedKFold(n_splits=5,shuffle=True,random_state=seed)
for train,test in Kfold.split(X=symA_train,y=y_train):
    ###############################################################################################

    input_shape=(322)
    base_network = create_base_network(input_shape)
    
    input_a = Input(shape=input_shape)
    input_b = Input(shape=input_shape)
    input_mesh_a = Input(shape=24)
    input_mesh_b = Input(shape=24)
    
    # because we re-use the same instance `base_network`,the weights of the network will be shared across the two branches
    processed_a = base_network(input_a)
    processed_b = base_network(input_b)
    

    eucl_model = create_euclLayer(processed_a.shape[-1])
    eucl_out = eucl_model([processed_a,processed_b]) 
    
    
    concat_fea = tf.keras.layers.concatenate([processed_a,input_mesh_a,processed_b,input_mesh_b],axis=-1)
    print("concat_fea的shape：",concat_fea.shape) #(None,32)
    MLP = create_MLP(concat_fea.shape[-1])
    MLP_out = MLP(concat_fea)
    
    model = Model(inputs=[input_a, input_b,input_mesh_a,input_mesh_b], outputs=[eucl_out,MLP_out])
    
    """L2"""
    for layer in model.layers:
        if hasattr(layer,'kernel_regularizer'):
            layer.kernel_regularizer = tf.keras.regularizers.l2(0.001)
        
    #####################################################################################################################

    print("-------------------------------------Kfold: {} iter-----------------------------------------".format(i))
    i+=1

    rms = RMSprop()
    callback = tf.keras.callbacks.EarlyStopping(monitor='loss',patience=5)
    
    my_loss = {"siamese":contrastive_loss,"mlp":tf.keras.losses.BinaryCrossentropy()}
    my_loss_weight = {"siamese":0,"mlp":1}
    my_metrics ={ "mlp":[tf.keras.metrics.BinaryAccuracy(),tf.keras.metrics.AUC()]} #tf.keras.metrics.Precision(top_k=len(rus_y)//100)
    
    model.compile(
#         loss= [tfa.losses.ContrastiveLoss(),tf.keras.losses.BinaryCrossentropy()],
#         loss_weights=[0.01,1],
#         metrics=[tf.keras.metrics.BinaryAccuracy(),tf.keras.metrics.AUC()],
        
        loss= my_loss,
        loss_weights=my_loss_weight,
        metrics=my_metrics,
        
        optimizer=rms,
    )
    
    # fit()中 shuffle=True
    model.fit([symA_train[train],symB_train[train],meshA_train[train],meshB_train[train]],
              [y_train[train],y_train[train]],
              
              batch_size=None,
              epochs=200,
              callbacks=callback,
              shuffle=True,
             )
    
    scores = model.evaluate([symA_train[test],symB_train[test],meshA_train[test],meshB_train[test]],
                            [y_train[test],y_train[test]],
                            verbose=1,
                            batch_size=32,
                            callbacks=callback)
    
    cvacc.append(scores[-2] * 100)
    cvauc_mlp.append(scores[-1] * 100)
    cvauc_sia.append(scores[-3] * 100)
    
    print("------ %s: %.2f%%\t ----- %s: %.2f%%" % 
           (model.metrics_names[-2],scores[-2]*100, model.metrics_names[-1],scores[-1]*100))
     
print("五折性能均值：-------- ave_acc: %.2f%% (+/- %.2f%%)\t ----- ave_auc_mlp:%.2f%% (+/- %.2f%%)" % 
      (np.mean(cvacc), np.std(cvacc),np.mean(cvauc_mlp), np.std(cvauc_mlp)))


concat_fea的shape： (None, 112)
-------------------------------------Kfold: 1 iter-----------------------------------------
Epoch 1/200
97/97 [==============================] - 0s 3ms/step - loss: 0.6703 - siamese_loss: 3.2520 - mlp_loss: 0.6703 - mlp_binary_accuracy: 0.5834 - mlp_auc_59: 0.6245
Epoch 2/200
97/97 [==============================] - 0s 3ms/step - loss: 0.6315 - siamese_loss: 3.8003 - mlp_loss: 0.6315 - mlp_binary_accuracy: 0.6357 - mlp_auc_59: 0.6940
Epoch 3/200
97/97 [==============================] - 0s 2ms/step - loss: 0.5928 - siamese_loss: 3.9250 - mlp_loss: 0.5928 - mlp_binary_accuracy: 0.6800 - mlp_auc_59: 0.7457
Epoch 4/200
97/97 [==============================] - 0s 3ms/step - loss: 0.5788 - siamese_loss: 4.0741 - mlp_loss: 0.5788 - mlp_binary_accuracy: 0.6975 - mlp_auc_59: 0.7622
Epoch 5/200
97/97 [==============================] - 0s 2ms/step - loss: 0.5621 - siamese_loss: 4.2057 - mlp_loss: 0.5621 - mlp_binary_accuracy: 0.7072 - mlp_auc_59: 0.7811
Epoch 6/200
9

97/97 [==============================] - 0s 2ms/step - loss: 0.5940 - siamese_loss: 3.2158 - mlp_loss: 0.5940 - mlp_binary_accuracy: 0.6803 - mlp_auc_60: 0.7490
Epoch 4/200
97/97 [==============================] - 0s 2ms/step - loss: 0.5762 - siamese_loss: 3.4715 - mlp_loss: 0.5762 - mlp_binary_accuracy: 0.7075 - mlp_auc_60: 0.7677
Epoch 5/200
97/97 [==============================] - 0s 2ms/step - loss: 0.5595 - siamese_loss: 3.6831 - mlp_loss: 0.5595 - mlp_binary_accuracy: 0.7136 - mlp_auc_60: 0.7841
Epoch 6/200
97/97 [==============================] - 0s 2ms/step - loss: 0.5520 - siamese_loss: 3.7292 - mlp_loss: 0.5520 - mlp_binary_accuracy: 0.7220 - mlp_auc_60: 0.7921
Epoch 7/200
97/97 [==============================] - 0s 2ms/step - loss: 0.5313 - siamese_loss: 3.8881 - mlp_loss: 0.5313 - mlp_binary_accuracy: 0.7304 - mlp_auc_60: 0.8107
Epoch 8/200
97/97 [==============================] - 0s 2ms/step - loss: 0.5245 - siamese_loss: 3.8493 - mlp_loss: 0.5245 - mlp_binary_accuracy: 0.

97/97 [==============================] - 0s 2ms/step - loss: 0.5700 - siamese_loss: 3.8852 - mlp_loss: 0.5700 - mlp_binary_accuracy: 0.7091 - mlp_auc_61: 0.7736
Epoch 5/200
97/97 [==============================] - 0s 2ms/step - loss: 0.5612 - siamese_loss: 3.8817 - mlp_loss: 0.5612 - mlp_binary_accuracy: 0.7220 - mlp_auc_61: 0.7812
Epoch 6/200
97/97 [==============================] - 0s 2ms/step - loss: 0.5364 - siamese_loss: 4.0943 - mlp_loss: 0.5364 - mlp_binary_accuracy: 0.7453 - mlp_auc_61: 0.8051
Epoch 7/200
97/97 [==============================] - 0s 2ms/step - loss: 0.5233 - siamese_loss: 4.1239 - mlp_loss: 0.5233 - mlp_binary_accuracy: 0.7434 - mlp_auc_61: 0.8170
Epoch 8/200
97/97 [==============================] - 0s 2ms/step - loss: 0.5148 - siamese_loss: 3.9992 - mlp_loss: 0.5148 - mlp_binary_accuracy: 0.7502 - mlp_auc_61: 0.8237
Epoch 9/200
97/97 [==============================] - 0s 2ms/step - loss: 0.5057 - siamese_loss: 4.0279 - mlp_loss: 0.5057 - mlp_binary_accuracy: 0.

97/97 [==============================] - 0s 2ms/step - loss: 0.5159 - siamese_loss: 3.7774 - mlp_loss: 0.5159 - mlp_binary_accuracy: 0.7519 - mlp_auc_62: 0.8248
Epoch 10/200
97/97 [==============================] - 0s 2ms/step - loss: 0.5107 - siamese_loss: 3.6479 - mlp_loss: 0.5107 - mlp_binary_accuracy: 0.7496 - mlp_auc_62: 0.8284
Epoch 11/200
97/97 [==============================] - 0s 2ms/step - loss: 0.4964 - siamese_loss: 3.7740 - mlp_loss: 0.4964 - mlp_binary_accuracy: 0.7628 - mlp_auc_62: 0.8393
Epoch 12/200
97/97 [==============================] - 0s 2ms/step - loss: 0.4968 - siamese_loss: 3.8576 - mlp_loss: 0.4968 - mlp_binary_accuracy: 0.7667 - mlp_auc_62: 0.8390
Epoch 13/200
97/97 [==============================] - 0s 2ms/step - loss: 0.4848 - siamese_loss: 3.9467 - mlp_loss: 0.4848 - mlp_binary_accuracy: 0.7719 - mlp_auc_62: 0.8478
Epoch 14/200
97/97 [==============================] - 0s 2ms/step - loss: 0.4756 - siamese_loss: 3.9939 - mlp_loss: 0.4756 - mlp_binary_accurac

97/97 [==============================] - 0s 3ms/step - loss: 0.4522 - siamese_loss: 4.7038 - mlp_loss: 0.4522 - mlp_binary_accuracy: 0.7961 - mlp_auc_63: 0.8646
Epoch 29/200
97/97 [==============================] - 0s 3ms/step - loss: 0.4533 - siamese_loss: 4.7822 - mlp_loss: 0.4533 - mlp_binary_accuracy: 0.8039 - mlp_auc_63: 0.8637
Epoch 30/200
97/97 [==============================] - 0s 2ms/step - loss: 0.4454 - siamese_loss: 4.8157 - mlp_loss: 0.4454 - mlp_binary_accuracy: 0.8081 - mlp_auc_63: 0.8674
Epoch 31/200
97/97 [==============================] - 0s 3ms/step - loss: 0.4365 - siamese_loss: 4.8619 - mlp_loss: 0.4365 - mlp_binary_accuracy: 0.8097 - mlp_auc_63: 0.8736
Epoch 32/200
97/97 [==============================] - 0s 2ms/step - loss: 0.4353 - siamese_loss: 5.0266 - mlp_loss: 0.4353 - mlp_binary_accuracy: 0.8181 - mlp_auc_63: 0.8778
Epoch 33/200
97/97 [==============================] - 0s 2ms/step - loss: 0.4323 - siamese_loss: 4.9395 - mlp_loss: 0.4323 - mlp_binary_accurac

97/97 [==============================] - 0s 3ms/step - loss: 0.1673 - siamese_loss: 5.6689 - mlp_loss: 0.1673 - mlp_binary_accuracy: 0.9399 - mlp_auc_63: 0.9801
Epoch 122/200
97/97 [==============================] - 0s 3ms/step - loss: 0.1741 - siamese_loss: 5.7036 - mlp_loss: 0.1741 - mlp_binary_accuracy: 0.9357 - mlp_auc_63: 0.9782
Epoch 123/200
97/97 [==============================] - 0s 3ms/step - loss: 0.1675 - siamese_loss: 5.7149 - mlp_loss: 0.1675 - mlp_binary_accuracy: 0.9409 - mlp_auc_63: 0.9789
Epoch 124/200
97/97 [==============================] - 0s 3ms/step - loss: 0.1702 - siamese_loss: 5.6010 - mlp_loss: 0.1702 - mlp_binary_accuracy: 0.9409 - mlp_auc_63: 0.9797
Epoch 125/200
97/97 [==============================] - 0s 3ms/step - loss: 0.1653 - siamese_loss: 5.7187 - mlp_loss: 0.1653 - mlp_binary_accuracy: 0.9425 - mlp_auc_63: 0.9805
Epoch 126/200
97/97 [==============================] - 0s 3ms/step - loss: 0.1686 - siamese_loss: 5.6833 - mlp_loss: 0.1686 - mlp_binary_ac

五折性能均值：-------- ave_acc: 76.65% (+/- 2.13%)	 ----- ave_auc_mlp:84.02% (+/- 1.70%)

## test3:only Mesh (siamese+mlp)

In [28]:
def create_siamese_test(input_shape):
    '''Base network to be shared (eq. to feature extraction).'''
    input = Input(shape=input_shape)
    
    x= Activation(activation='relu')(Dense(48)(input))
    x= Activation(activation='relu')(Dense(32)(x))
    x= Activation(activation='relu')(Dense(16)(x))
    x= Activation(activation='sigmoid')(Dense(8)(x))
    
    return Model(input, x,name="siamese")

def create_MLP_test(input_shape):
    
    input = Input(shape=input_shape)
    x= Activation(activation='relu')(Dense(16)(input))
    
    x= Activation(activation='relu')(Dense(8)(x))
    
    x= Dense(1,activation='sigmoid')(x)
    
    return Model(input, x ,name="mlp_test")

def create_euclLayer_test(input_shape):
    pair1 = Input(shape=(input_shape))
    pair2 = Input(shape=(input_shape))
              
    x = euclLayer()(pair1,pair2)
    x = Activation(activation='relu')(x)
#     x = Activation(activation='sigmoid')(x)
    
    return Model(inputs=[pair1,pair2],outputs=x,name="siamese_test")

In [29]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics  import roc_curve,auc,roc_auc_score
from imblearn.under_sampling import RandomUnderSampler

mesh_label_A = np.array(mesh_label_A)
mesh_label_B = np.array(mesh_label_B)
y = np.array(y)

cvacc,cvauc_mlp,cvpre,cvpre_top,cvrec,cvauc_sia=[],[],[],[],[],[]
i =1

y=np.array(y)
disA_fea_mat=np.array(mesh_label_A)
disB_fea_mat=np.array(mesh_label_B)

rus = RandomUnderSampler(sampling_strategy=1,random_state=11)
rus_dfmA,rus_y = rus.fit_resample(X=disA_fea_mat,y=y)
rus_dfmB,rus_y = rus.fit_resample(X=disB_fea_mat,y=y)
print('rus_y的长度:',len(rus_y))


seed = 201202
np.random.seed(seed) # This is necessary for starting Numpy generated random numbers in a well-defined initial state.
python_random.seed(seed)# This is necessary for starting core Python generated random numbers in a well-defined state.
tf.random.set_seed(seed)

"""10-fold"""
Kfold = StratifiedKFold(n_splits=10,shuffle=True,random_state=seed)
for train,test in Kfold.split(X=rus_dfmA,y=rus_y):
    ###############################################################################################

    input_shape=(24)
    base_network = create_siamese_test(input_shape)
    
    input_a = Input(shape=input_shape)
    input_b = Input(shape=input_shape)
    # because we re-use the same instance `base_network`,the weights of the network will be shared across the two branches
    processed_a = base_network(input_a)
    processed_b = base_network(input_b)
    

    eucl_model = create_euclLayer_test(processed_a.shape[-1])
    eucl_out = eucl_model([processed_a,processed_b]) 
      

    concat_fea = tf.keras.layers.concatenate([processed_a,processed_b],axis=-1)
    print("concat_fea的shape：",concat_fea.shape)
    MLP = create_MLP_test(concat_fea.shape[-1])
    MLP_out = MLP(concat_fea)
    
    model = Model(inputs=[input_a, input_b], outputs=[eucl_out,MLP_out])
    
    """L2"""
    for layer in model.layers:
        if hasattr(layer,'kernel_regularizer'):
            layer.kernel_regularizer = tf.keras.regularizers.l2(0.001)
        
    #####################################################################################################################
    
    print("-------------------------------------Kfold: {} iter-----------------------------------------".format(i))
    i+=1

    rms = RMSprop()
    callback = tf.keras.callbacks.EarlyStopping(monitor='loss',patience=5)
    
    my_loss = {"siamese_test":contrastive_loss,"mlp_test":tf.keras.losses.BinaryCrossentropy()}
    my_loss_weight = {"siamese_test":0,"mlp_test":1}
    my_metrics ={ "mlp_test":[tf.keras.metrics.BinaryAccuracy(),tf.keras.metrics.AUC()]} #tf.keras.metrics.Precision(top_k=len(rus_y)//100)
    
    model.compile(
#         loss= [tfa.losses.ContrastiveLoss(),tf.keras.losses.BinaryCrossentropy()],
#         loss_weights=[0.01,1],
#         metrics=[tf.keras.metrics.BinaryAccuracy(),tf.keras.metrics.AUC()],
        
        loss= my_loss,
        loss_weights=my_loss_weight,
        metrics=my_metrics,
        
        optimizer=rms,
    )
    
    # fit()中 shuffle=True
    model.fit([rus_dfmA[train],rus_dfmB[train]],[rus_y[train],rus_y[train]],
              batch_size=None,
              epochs=200,
              callbacks=callback,
              shuffle=True)
    
    scores = model.evaluate([rus_dfmA[test],rus_dfmB[test]],[rus_y[test],rus_y[test]],
                            verbose=1,
                            batch_size=len(rus_y[test]),
                            callbacks=callback)
    
    cvacc.append(scores[-2] * 100)
    cvauc_mlp.append(scores[-1] * 100)
    cvauc_sia.append(scores[-3] * 100)
    
    print("------ %s: %.2f%%\t ----- %s: %.2f%%" % 
           (model.metrics_names[-2],scores[-2]*100, model.metrics_names[-1],scores[-1]*100))
     
#print("ave_auc_keras: %.2f%% (+/- %.2f%%)\t ave_topk_pre: %.2f%% (+/- %.2f%%)\t ave_auc:%.2f%% (+/- %.2f%%)" % 
print("十折性能均值：-------- ave_acc: %.2f%% (+/- %.2f%%)\t ----- ave_auc_mlp:%.2f%% (+/- %.2f%%)" % 
      (np.mean(cvacc), np.std(cvacc),np.mean(cvauc_mlp), np.std(cvauc_mlp)))


rus_y的长度: 4836
concat_fea的shape： (None, 16)
-------------------------------------Kfold: 1 iter-----------------------------------------


'PYTHONHASHSEED' 不是内部或外部命令，也不是可运行的程序
或批处理文件。


Epoch 1/200
136/136 [==============================] - 0s 1ms/step - loss: 0.6866 - siamese_test_loss: 0.4158 - mlp_test_loss: 0.6866 - mlp_test_binary_accuracy: 0.5338 - mlp_test_auc_14: 0.5785
Epoch 2/200
136/136 [==============================] - 0s 1ms/step - loss: 0.6726 - siamese_test_loss: 0.3599 - mlp_test_loss: 0.6726 - mlp_test_binary_accuracy: 0.5864 - mlp_test_auc_14: 0.6383
Epoch 3/200
136/136 [==============================] - 0s 1ms/step - loss: 0.6610 - siamese_test_loss: 0.3408 - mlp_test_loss: 0.6610 - mlp_test_binary_accuracy: 0.6103 - mlp_test_auc_14: 0.6612
Epoch 4/200
136/136 [==============================] - 0s 1ms/step - loss: 0.6498 - siamese_test_loss: 0.3408 - mlp_test_loss: 0.6498 - mlp_test_binary_accuracy: 0.6326 - mlp_test_auc_14: 0.6730
Epoch 5/200
136/136 [==============================] - 0s 1ms/step - loss: 0.6399 - siamese_test_loss: 0.3397 - mlp_test_loss: 0.6399 - mlp_test_binary_accuracy: 0.6448 - mlp_test_auc_14: 0.6940
Epoch 6/200
136/136 [====

136/136 [==============================] - 0s 2ms/step - loss: 0.5587 - siamese_test_loss: 0.4034 - mlp_test_loss: 0.5587 - mlp_test_binary_accuracy: 0.7006 - mlp_test_auc_14: 0.7785
Epoch 84/200
136/136 [==============================] - 0s 1ms/step - loss: 0.5587 - siamese_test_loss: 0.3945 - mlp_test_loss: 0.5587 - mlp_test_binary_accuracy: 0.6962 - mlp_test_auc_14: 0.7768
Epoch 85/200
136/136 [==============================] - 0s 1ms/step - loss: 0.5575 - siamese_test_loss: 0.3942 - mlp_test_loss: 0.5575 - mlp_test_binary_accuracy: 0.7107 - mlp_test_auc_14: 0.7800
Epoch 86/200
136/136 [==============================] - 0s 1ms/step - loss: 0.5588 - siamese_test_loss: 0.4016 - mlp_test_loss: 0.5588 - mlp_test_binary_accuracy: 0.7057 - mlp_test_auc_14: 0.7777
Epoch 87/200
136/136 [==============================] - 0s 1ms/step - loss: 0.5579 - siamese_test_loss: 0.4070 - mlp_test_loss: 0.5579 - mlp_test_binary_accuracy: 0.6981 - mlp_test_auc_14: 0.7787
Epoch 88/200
136/136 [===========

1/1 [==============================] - 0s 983us/step - loss: 0.6083 - siamese_test_loss: 0.4927 - mlp_test_loss: 0.6083 - mlp_test_binary_accuracy: 0.6963 - mlp_test_auc_14: 0.7542
------ mlp_test_binary_accuracy: 69.63%	 ----- mlp_test_auc_14: 75.42%
concat_fea的shape： (None, 16)
-------------------------------------Kfold: 2 iter-----------------------------------------
Epoch 1/200
136/136 [==============================] - 0s 1ms/step - loss: 0.6905 - siamese_test_loss: 0.4033 - mlp_test_loss: 0.6905 - mlp_test_binary_accuracy: 0.5280 - mlp_test_auc_15: 0.5444
Epoch 2/200
136/136 [==============================] - 0s 1ms/step - loss: 0.6746 - siamese_test_loss: 0.3269 - mlp_test_loss: 0.6746 - mlp_test_binary_accuracy: 0.5967 - mlp_test_auc_15: 0.6388
Epoch 3/200
136/136 [==============================] - 0s 1ms/step - loss: 0.6530 - siamese_test_loss: 0.3266 - mlp_test_loss: 0.6530 - mlp_test_binary_accuracy: 0.6213 - mlp_test_auc_15: 0.6670
Epoch 4/200
136/136 [=====================

...

136/136 [==============================] - 0s 1ms/step - loss: 0.5290 - siamese_test_loss: 0.6024 - mlp_test_loss: 0.5290 - mlp_test_binary_accuracy: 0.7307 - mlp_test_auc_19: 0.8093
Epoch 118/200
136/136 [==============================] - 0s 1ms/step - loss: 0.5286 - siamese_test_loss: 0.5969 - mlp_test_loss: 0.5286 - mlp_test_binary_accuracy: 0.7316 - mlp_test_auc_19: 0.8095
Epoch 119/200
136/136 [==============================] - 0s 1ms/step - loss: 0.5273 - siamese_test_loss: 0.6041 - mlp_test_loss: 0.5273 - mlp_test_binary_accuracy: 0.7295 - mlp_test_auc_19: 0.8108
Epoch 120/200
136/136 [==============================] - 0s 1ms/step - loss: 0.5272 - siamese_test_loss: 0.6032 - mlp_test_loss: 0.5272 - mlp_test_binary_accuracy: 0.7325 - mlp_test_auc_19: 0.8104
Epoch 121/200
136/136 [==============================] - 0s 1ms/step - loss: 0.5265 - siamese_test_loss: 0.6079 - mlp_test_loss: 0.5265 - mlp_test_binary_accuracy: 0.7282 - mlp_test_auc_19: 0.8108
Epoch 122/200
136/136 [======

137/137 [==============================] - 0s 1ms/step - loss: 0.6002 - siamese_test_loss: 0.3642 - mlp_test_loss: 0.6002 - mlp_test_binary_accuracy: 0.6595 - mlp_test_auc_20: 0.7314
Epoch 11/200
137/137 [==============================] - 0s 1ms/step - loss: 0.5987 - siamese_test_loss: 0.3662 - mlp_test_loss: 0.5987 - mlp_test_binary_accuracy: 0.6621 - mlp_test_auc_20: 0.7328
Epoch 12/200
137/137 [==============================] - 0s 1ms/step - loss: 0.5978 - siamese_test_loss: 0.3667 - mlp_test_loss: 0.5978 - mlp_test_binary_accuracy: 0.6653 - mlp_test_auc_20: 0.7349
Epoch 13/200
137/137 [==============================] - 0s 1ms/step - loss: 0.5963 - siamese_test_loss: 0.3664 - mlp_test_loss: 0.5963 - mlp_test_binary_accuracy: 0.6664 - mlp_test_auc_20: 0.7361
Epoch 14/200
137/137 [==============================] - 0s 1ms/step - loss: 0.5938 - siamese_test_loss: 0.3677 - mlp_test_loss: 0.5938 - mlp_test_binary_accuracy: 0.6655 - mlp_test_auc_20: 0.7384
Epoch 15/200
137/137 [===========

137/137 [==============================] - 0s 1ms/step - loss: 0.5701 - siamese_test_loss: 0.4134 - mlp_test_loss: 0.5701 - mlp_test_binary_accuracy: 0.7000 - mlp_test_auc_21: 0.7694
Epoch 63/200
137/137 [==============================] - 0s 1ms/step - loss: 0.5715 - siamese_test_loss: 0.4086 - mlp_test_loss: 0.5715 - mlp_test_binary_accuracy: 0.6947 - mlp_test_auc_21: 0.7680
Epoch 64/200
137/137 [==============================] - 0s 1ms/step - loss: 0.5700 - siamese_test_loss: 0.4090 - mlp_test_loss: 0.5700 - mlp_test_binary_accuracy: 0.6968 - mlp_test_auc_21: 0.7694
Epoch 65/200
137/137 [==============================] - 0s 1ms/step - loss: 0.5701 - siamese_test_loss: 0.4089 - mlp_test_loss: 0.5701 - mlp_test_binary_accuracy: 0.7014 - mlp_test_auc_21: 0.7689
Epoch 66/200
137/137 [==============================] - 0s 1ms/step - loss: 0.5672 - siamese_test_loss: 0.4164 - mlp_test_loss: 0.5672 - mlp_test_binary_accuracy: 0.6981 - mlp_test_auc_21: 0.7720
Epoch 67/200
137/137 [===========

137/137 [==============================] - 0s 1ms/step - loss: 0.5351 - siamese_test_loss: 0.4918 - mlp_test_loss: 0.5351 - mlp_test_binary_accuracy: 0.7308 - mlp_test_auc_21: 0.8036
Epoch 145/200
137/137 [==============================] - 0s 1ms/step - loss: 0.5345 - siamese_test_loss: 0.4881 - mlp_test_loss: 0.5345 - mlp_test_binary_accuracy: 0.7248 - mlp_test_auc_21: 0.8030
Epoch 146/200
137/137 [==============================] - 0s 1ms/step - loss: 0.5338 - siamese_test_loss: 0.4910 - mlp_test_loss: 0.5338 - mlp_test_binary_accuracy: 0.7207 - mlp_test_auc_21: 0.8042
Epoch 147/200
137/137 [==============================] - 0s 1ms/step - loss: 0.5336 - siamese_test_loss: 0.4915 - mlp_test_loss: 0.5336 - mlp_test_binary_accuracy: 0.7250 - mlp_test_auc_21: 0.8043
Epoch 148/200
137/137 [==============================] - 0s 1ms/step - loss: 0.5325 - siamese_test_loss: 0.4901 - mlp_test_loss: 0.5325 - mlp_test_binary_accuracy: 0.7271 - mlp_test_auc_21: 0.8058
Epoch 149/200
137/137 [======

137/137 [==============================] - 0s 1ms/step - loss: 0.5933 - siamese_test_loss: 0.3691 - mlp_test_loss: 0.5933 - mlp_test_binary_accuracy: 0.6692 - mlp_test_auc_22: 0.7406
Epoch 22/200
137/137 [==============================] - 0s 1ms/step - loss: 0.5917 - siamese_test_loss: 0.3689 - mlp_test_loss: 0.5917 - mlp_test_binary_accuracy: 0.6713 - mlp_test_auc_22: 0.7433
Epoch 23/200
137/137 [==============================] - 0s 2ms/step - loss: 0.5919 - siamese_test_loss: 0.3722 - mlp_test_loss: 0.5919 - mlp_test_binary_accuracy: 0.6745 - mlp_test_auc_22: 0.7431
Epoch 24/200
137/137 [==============================] - 0s 1ms/step - loss: 0.5919 - siamese_test_loss: 0.3698 - mlp_test_loss: 0.5919 - mlp_test_binary_accuracy: 0.6736 - mlp_test_auc_22: 0.7428
Epoch 25/200
137/137 [==============================] - 0s 1ms/step - loss: 0.5917 - siamese_test_loss: 0.3782 - mlp_test_loss: 0.5917 - mlp_test_binary_accuracy: 0.6754 - mlp_test_auc_22: 0.7436
Epoch 26/200
137/137 [===========

### results

-------- ave_acc: 67.84% (+/- 3.25%)	 ----- ave_auc_mlp:75.32% (+/- 2.61%)

## test4: only Mesh (MLP)

In [65]:
mesh_label_concat = pd.concat([mesh_label_A.reset_index(drop=True),mesh_label_B.reset_index(drop=True)],axis=1)
print(mesh_label_concat.shape)

(35245, 48)


In [72]:
# pd.DataFrame(mesh_label_concat).to_csv("mesh_concat.csv")

In [73]:
from tensorflow.keras.models import Sequential
i=1
cvacc=[]
cvauc=[]

mesh_label_concat=np.array(mesh_label_concat)

seed = 22

import os
os.environ['PYTHONHASHSEED'] = '0'
# np.random.seed(seed)
python_random.seed(seed)
tf.random.set_seed(seed)

rus = RandomUnderSampler(sampling_strategy=1)

rus_x,rus_y = rus.fit_resample(X=mesh_label_concat,y=y)
print('rus_y的长度:',len(rus_y))


Kfold = StratifiedKFold(n_splits=10,shuffle=True,random_state=seed)
for train,test in Kfold.split(X=rus_x,y=rus_y):
    print("--------------Kfold: {} iter".format(i))
    i+=1
    rms = RMSprop()
    callback = tf.keras.callbacks.EarlyStopping(monitor='loss',patience=2)
    ##################################################################################################
    model = Sequential()
    model.add(Dense(32, activation='relu', input_dim=48))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(16, activation='relu'))
    model.add(Dense(8, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    
    """L2"""
    for layer in model.layers:
        if hasattr(layer,'kernel_regularizer'):
            layer.kernel_regularizer = tf.keras.regularizers.l2(0.001)
    ##################################################################################################
    model.compile(
            loss=tf.keras.losses.BinaryCrossentropy(),
            optimizer=rms,
            metrics=[tf.keras.metrics.BinaryAccuracy(),tf.keras.metrics.AUC()]
    )
    
    model.fit(rus_x[train],rus_y[train],
              #class_weight={0:1,1:1},
              batch_size=None,
              epochs=100,
              callbacks=callback,
              shuffle=True)
    
    scores = model.evaluate(rus_x[test],rus_y[test],
                            verbose=1,
                            batch_size=None,
                            callbacks=callback)
    
    cvacc.append(scores[1] * 100)
    cvauc.append(scores[2] * 100)
    
    print(" %s: %.2f%%\t ------ %s: %.2f%%\t " % 
           (model.metrics_names[1],scores[1]*100,model.metrics_names[2],scores[2]*100))
    
#print("ave_auc_keras: %.2f%% (+/- %.2f%%)\t ave_topk_pre: %.2f%% (+/- %.2f%%)\t ave_auc:%.2f%% (+/- %.2f%%)" % 
print("ave_acc: %.2f%% (+/- %.2f%%)\t ------ ave_auc: %.2f%% (+/- %.2f%%)\t " % 
      (np.mean(cvacc), np.std(cvacc),np.mean(cvauc), np.std(cvauc)))

print(model.summary())

rus_y的长度: 4836
--------------Kfold: 1 iter
Epoch 1/100
136/136 [==============================] - 0s 1ms/step - loss: 0.6778 - binary_accuracy: 0.5832 - auc_167: 0.6205
Epoch 2/100
136/136 [==============================] - 0s 927us/step - loss: 0.6374 - binary_accuracy: 0.6372 - auc_167: 0.6935
Epoch 3/100
136/136 [==============================] - 0s 943us/step - loss: 0.6108 - binary_accuracy: 0.6620 - auc_167: 0.7234
Epoch 4/100
136/136 [==============================] - 0s 890us/step - loss: 0.5909 - binary_accuracy: 0.6739 - auc_167: 0.7475
Epoch 5/100
136/136 [==============================] - 0s 927us/step - loss: 0.5738 - binary_accuracy: 0.6907 - auc_167: 0.7657
Epoch 6/100
136/136 [==============================] - 0s 941us/step - loss: 0.5622 - binary_accuracy: 0.6967 - auc_167: 0.7776
Epoch 7/100
136/136 [==============================] - 0s 969us/step - loss: 0.5499 - binary_accuracy: 0.7119 - auc_167: 0.7902
Epoch 8/100
136/136 [==============================] - 0s 963us

136/136 [==============================] - 0s 1ms/step - loss: 0.4372 - binary_accuracy: 0.7822 - auc_168: 0.8753
Epoch 40/100
136/136 [==============================] - 0s 1ms/step - loss: 0.4350 - binary_accuracy: 0.7842 - auc_168: 0.8766
Epoch 41/100
136/136 [==============================] - 0s 1ms/step - loss: 0.4327 - binary_accuracy: 0.7884 - auc_168: 0.8778
Epoch 42/100
136/136 [==============================] - 0s 999us/step - loss: 0.4329 - binary_accuracy: 0.7870 - auc_168: 0.8777
Epoch 43/100
136/136 [==============================] - 0s 889us/step - loss: 0.4306 - binary_accuracy: 0.7854 - auc_168: 0.8790
Epoch 44/100
136/136 [==============================] - 0s 904us/step - loss: 0.4282 - binary_accuracy: 0.7877 - auc_168: 0.8806
Epoch 45/100
136/136 [==============================] - 0s 911us/step - loss: 0.4287 - binary_accuracy: 0.7911 - auc_168: 0.8798
Epoch 46/100
136/136 [==============================] - 0s 911us/step - loss: 0.4263 - binary_accuracy: 0.7902 - auc

136/136 [==============================] - 0s 869us/step - loss: 0.4365 - binary_accuracy: 0.7824 - auc_170: 0.8757
Epoch 37/100
136/136 [==============================] - 0s 815us/step - loss: 0.4324 - binary_accuracy: 0.7812 - auc_170: 0.8774
Epoch 38/100
136/136 [==============================] - 0s 841us/step - loss: 0.4318 - binary_accuracy: 0.7812 - auc_170: 0.8780
Epoch 39/100
136/136 [==============================] - 0s 921us/step - loss: 0.4285 - binary_accuracy: 0.7835 - auc_170: 0.8803
Epoch 40/100
136/136 [==============================] - 0s 850us/step - loss: 0.4288 - binary_accuracy: 0.7847 - auc_170: 0.8796
Epoch 41/100
136/136 [==============================] - 0s 872us/step - loss: 0.4265 - binary_accuracy: 0.7863 - auc_170: 0.8804
Epoch 42/100
136/136 [==============================] - 0s 808us/step - loss: 0.4258 - binary_accuracy: 0.7831 - auc_170: 0.8811
Epoch 43/100
136/136 [==============================] - 0s 879us/step - loss: 0.4208 - binary_accuracy: 0.7907

136/136 [==============================] - 0s 986us/step - loss: 0.3818 - binary_accuracy: 0.8109 - auc_171: 0.9041
Epoch 88/100
136/136 [==============================] - 0s 751us/step - loss: 0.3813 - binary_accuracy: 0.8125 - auc_171: 0.9050
Epoch 89/100
136/136 [==============================] - 0s 815us/step - loss: 0.3806 - binary_accuracy: 0.8070 - auc_171: 0.9046
Epoch 90/100
136/136 [==============================] - 0s 1ms/step - loss: 0.3790 - binary_accuracy: 0.8146 - auc_171: 0.9058
Epoch 91/100
136/136 [==============================] - 0s 900us/step - loss: 0.3787 - binary_accuracy: 0.8123 - auc_171: 0.9057
Epoch 92/100
136/136 [==============================] - 0s 789us/step - loss: 0.3760 - binary_accuracy: 0.8153 - auc_171: 0.9075
Epoch 93/100
136/136 [==============================] - 0s 861us/step - loss: 0.3797 - binary_accuracy: 0.8114 - auc_171: 0.9052
Epoch 94/100
16/16 [==============================] - 0s 331us/step - loss: 0.6927 - binary_accuracy: 0.7066 - a

137/137 [==============================] - 0s 811us/step - loss: 0.4322 - binary_accuracy: 0.7868 - auc_173: 0.8779
Epoch 45/100
137/137 [==============================] - 0s 828us/step - loss: 0.4337 - binary_accuracy: 0.7834 - auc_173: 0.8768
Epoch 46/100
16/16 [==============================] - 0s 872us/step - loss: 0.5847 - binary_accuracy: 0.7039 - auc_173: 0.7849
 binary_accuracy: 70.39%	 ------ auc_173: 78.49%	 
--------------Kfold: 8 iter
Epoch 1/100
137/137 [==============================] - 0s 948us/step - loss: 0.6761 - binary_accuracy: 0.5881 - auc_174: 0.6318
Epoch 2/100
137/137 [==============================] - 0s 948us/step - loss: 0.6317 - binary_accuracy: 0.6290 - auc_174: 0.6990
Epoch 3/100
137/137 [==============================] - 0s 897us/step - loss: 0.6043 - binary_accuracy: 0.6586 - auc_174: 0.7276
Epoch 4/100
137/137 [==============================] - 0s 1ms/step - loss: 0.5891 - binary_accuracy: 0.6759 - auc_174: 0.7468
Epoch 5/100
137/137 [==================

137/137 [==============================] - 0s 1ms/step - loss: 0.4556 - binary_accuracy: 0.7758 - auc_176: 0.8647
Epoch 31/100
137/137 [==============================] - 0s 1ms/step - loss: 0.4533 - binary_accuracy: 0.7756 - auc_176: 0.8660
Epoch 32/100
137/137 [==============================] - 0s 1ms/step - loss: 0.4490 - binary_accuracy: 0.7827 - auc_176: 0.8688
Epoch 33/100
137/137 [==============================] - 0s 1ms/step - loss: 0.4475 - binary_accuracy: 0.7847 - auc_176: 0.8696
Epoch 34/100
137/137 [==============================] - 0s 1ms/step - loss: 0.4458 - binary_accuracy: 0.7841 - auc_176: 0.8705
Epoch 35/100
137/137 [==============================] - 0s 1ms/step - loss: 0.4450 - binary_accuracy: 0.7797 - auc_176: 0.8709
Epoch 36/100
137/137 [==============================] - 0s 1ms/step - loss: 0.4414 - binary_accuracy: 0.7831 - auc_176: 0.8734
Epoch 37/100
137/137 [==============================] - 0s 1ms/step - loss: 0.4374 - binary_accuracy: 0.7903 - auc_176: 0.87

### results

ave_acc: 69.93% (+/- 1.77%)	 ------ ave_auc: 76.81% (+/- 1.83%)

## test5:Mesh+sym（Siamese）
splicing disease symptom and mesh features of diseasea pairs

In [15]:
confeaA = pd.read_csv("concat_mesh_label/con_feaA.csv",index_col=0)
print(confeaA.shape)
confeaB = pd.read_csv("concat_mesh_label/con_feaB.csv",index_col=0)

(35245, 346)


In [17]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics  import roc_curve,auc,roc_auc_score
from imblearn.under_sampling import RandomUnderSampler

cvacc=[]
cvauc=[]
cvauc_keras=[]
cvpre=[]
cvpre_top_keras =[]
cvpre_top=[]
cvrec=[]
i =1

y=np.array(y)
disA_fea_mat=np.array(confeaA)
disB_fea_mat=np.array(confeaB)


rus = RandomUnderSampler(sampling_strategy=1,random_state=11)
rus_dfmA,rus_y = rus.fit_resample(X=disA_fea_mat,y=y)
rus_dfmB,rus_y = rus.fit_resample(X=disB_fea_mat,y=y)
print('rus_y的长度:',len(rus_y))


seed = 201202
!PYTHONHASHSEED=0 
np.random.seed(seed) # This is necessary for starting Numpy generated random numbers in a well-defined initial state.
python_random.seed(seed)# This is necessary for starting core Python generated random numbers in a well-defined state.
tf.random.set_seed(seed)

"""10-fold"""
Kfold = StratifiedKFold(n_splits=10,shuffle=True,random_state=seed)
for train,test in Kfold.split(X=rus_dfmA,y=rus_y):
    print("--------------Kfold: {} iter".format(i))
    i+=1

    rms = RMSprop()
    callback = tf.keras.callbacks.EarlyStopping(monitor='loss',patience=5)
    
    ####################################################################################

    input_shape=(346)
    base_network = create_base_network(input_shape)
    
    input_a = Input(shape=input_shape)
    input_b = Input(shape=input_shape)
    # because we re-use the same instance `base_network`,the weights of the network will be shared across the two branches
    processed_a = base_network(input_a)
    processed_b = base_network(input_b)
    

    output_distance = euclLayer()(processed_a,processed_b)
    final_out= 1-tf.keras.activations.tanh(output_distance)

    
    model = Model([input_a, input_b], final_out)
    
    ####################################################################################
    model.compile(
            loss=tf.keras.losses.BinaryCrossentropy(),
#             loss= contrastive_loss,
#             loss= tfa.losses.ContrastiveLoss(),
            optimizer=Adam(),
            metrics=[tf.keras.metrics.BinaryAccuracy()]
            #metrics=[tf.keras.metrics.BinaryAccuracy(),tf.keras.metrics.AUC()]
    )
    
    # fit()中 shuffle=True
    model.fit([rus_dfmA[train],rus_dfmB[train]],rus_y[train],
              class_weight={0:1,1:1},
              batch_size=None,
              epochs=200,
              callbacks=callback,
              shuffle=True)
    
    scores = model.evaluate([rus_dfmA[test],rus_dfmB[test]],rus_y[test],
                            verbose=1,
                            batch_size=len(rus_y[test]),
                            callbacks=callback)
    
    """###########################AUC,topk Precision#############################"""
    y_pred = model.predict([rus_dfmA[test],rus_dfmB[test]],
                          batch_size = len(rus_y[test]))
    print("y_pred长度：",len(y_pred))
    
    roc_auc = roc_auc_score(rus_y[test],y_pred)
    
    #top_Precision
    top = 100
    y_pred = y_pred.flatten()
    topk_pre = top_precision(rus_y[test],y_pred,top)
    #print(rus_y[test])
    #print(y_pred)
    
    cvauc.append(roc_auc * 100)
    cvpre_top.append(topk_pre * 100)
    cvacc.append(scores[1] * 100)
    #cvpre_top_keras.append(scores[2] * 100)
    #cvauc_keras.append(scores[1] * 100)
    
    print(" %s: %.2f%%\t %s: %.2f%%\t %s: %.2f%%\t" % 
           (model.metrics_names[1],scores[1]*100,"topk_pre",topk_pre*100, "AUC_all",roc_auc*100))
     
#print("ave_auc_keras: %.2f%% (+/- %.2f%%)\t ave_topk_pre: %.2f%% (+/- %.2f%%)\t ave_auc:%.2f%% (+/- %.2f%%)" % 
print("ave_acc: %.2f%% (+/- %.2f%%)\t ave_topk_pre: %.2f%% (+/- %.2f%%)\t ave_auc:%.2f%% (+/- %.2f%%)" % 
      (np.mean(cvacc), np.std(cvacc),np.mean(cvpre_top), np.std(cvpre_top),np.mean(cvauc), np.std(cvauc)))

rus_y的长度: 4836
--------------Kfold: 1 iter


'PYTHONHASHSEED' 不是内部或外部命令，也不是可运行的程序
或批处理文件。


Epoch 1/200
136/136 [==============================] - 0s 2ms/step - loss: 1.1800 - binary_accuracy: 0.4998
Epoch 2/200
136/136 [==============================] - 0s 2ms/step - loss: 0.9645 - binary_accuracy: 0.5062
Epoch 3/200
136/136 [==============================] - 0s 2ms/step - loss: 0.9236 - binary_accuracy: 0.5221
Epoch 4/200
136/136 [==============================] - 0s 2ms/step - loss: 0.8810 - binary_accuracy: 0.5257
Epoch 5/200
136/136 [==============================] - 0s 2ms/step - loss: 0.8539 - binary_accuracy: 0.5618
Epoch 6/200
136/136 [==============================] - 0s 2ms/step - loss: 0.8712 - binary_accuracy: 0.5558
Epoch 7/200
136/136 [==============================] - 0s 2ms/step - loss: 0.8573 - binary_accuracy: 0.5492
Epoch 8/200
136/136 [==============================] - 0s 2ms/step - loss: 0.8289 - binary_accuracy: 0.5724
Epoch 9/200
136/136 [==============================] - 0s 2ms/step - loss: 0.7878 - binary_accuracy: 0.5767
Epoch 10/200
136/136 [======

1/1 [==============================] - 0s 0s/step - loss: 0.6765 - binary_accuracy: 0.6260
y_pred长度： 484
 binary_accuracy: 62.60%	 topk_pre: 28.00%	 AUC_all: 67.87%	
--------------Kfold: 4 iter
Epoch 1/200
136/136 [==============================] - 0s 2ms/step - loss: 1.1670 - binary_accuracy: 0.5119
Epoch 2/200
136/136 [==============================] - 0s 2ms/step - loss: 0.9337 - binary_accuracy: 0.5154
Epoch 3/200
136/136 [==============================] - 0s 2ms/step - loss: 0.9119 - binary_accuracy: 0.5179
Epoch 4/200
136/136 [==============================] - 0s 2ms/step - loss: 0.8938 - binary_accuracy: 0.5400
Epoch 5/200
136/136 [==============================] - 0s 2ms/step - loss: 0.8707 - binary_accuracy: 0.5308
Epoch 6/200
136/136 [==============================] - 0s 2ms/step - loss: 0.8729 - binary_accuracy: 0.5349
Epoch 7/200
136/136 [==============================] - 0s 2ms/step - loss: 0.8546 - binary_accuracy: 0.5437
Epoch 8/200
136/136 [=============================

136/136 [==============================] - 0s 1ms/step - loss: 0.6285 - binary_accuracy: 0.6756
Epoch 51/200
136/136 [==============================] - 0s 1ms/step - loss: 0.6103 - binary_accuracy: 0.6875
Epoch 52/200
136/136 [==============================] - 0s 1ms/step - loss: 0.6197 - binary_accuracy: 0.6756
Epoch 53/200
136/136 [==============================] - 0s 1ms/step - loss: 0.6286 - binary_accuracy: 0.6739
Epoch 54/200
136/136 [==============================] - 0s 1ms/step - loss: 0.6222 - binary_accuracy: 0.6781
Epoch 55/200
136/136 [==============================] - 0s 1ms/step - loss: 0.6271 - binary_accuracy: 0.6737
Epoch 56/200
1/1 [==============================] - 0s 984us/step - loss: 0.6533 - binary_accuracy: 0.6674
y_pred长度： 484
 binary_accuracy: 66.74%	 topk_pre: 26.00%	 AUC_all: 72.22%	
--------------Kfold: 7 iter
Epoch 1/200
137/137 [==============================] - 0s 2ms/step - loss: 1.0866 - binary_accuracy: 0.5072
Epoch 2/200
137/137 [====================

137/137 [==============================] - 0s 2ms/step - loss: 0.7150 - binary_accuracy: 0.6283
Epoch 25/200
137/137 [==============================] - 0s 2ms/step - loss: 0.6910 - binary_accuracy: 0.6402
Epoch 26/200
137/137 [==============================] - 0s 2ms/step - loss: 0.7047 - binary_accuracy: 0.6435
Epoch 27/200
137/137 [==============================] - 0s 2ms/step - loss: 0.7061 - binary_accuracy: 0.6377
Epoch 28/200
137/137 [==============================] - 0s 2ms/step - loss: 0.6890 - binary_accuracy: 0.6407
Epoch 29/200
137/137 [==============================] - 0s 2ms/step - loss: 0.6827 - binary_accuracy: 0.6391
Epoch 30/200
137/137 [==============================] - 0s 2ms/step - loss: 0.6855 - binary_accuracy: 0.6561
Epoch 31/200
137/137 [==============================] - 0s 2ms/step - loss: 0.6794 - binary_accuracy: 0.6593
Epoch 32/200
137/137 [==============================] - 0s 2ms/step - loss: 0.6730 - binary_accuracy: 0.6612
Epoch 33/200
137/137 [==========

### results

ave_acc: 63.75% (+/- 2.49%)	 ave_topk_pre: 26.40% (+/- 4.36%)	 ave_auc:69.16% (+/- 2.33%)